In [1]:
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, accuracy_score, classification_report, precision_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier

In [2]:
#Chargement du DF
data = pd.read_excel('Copie de Coeur.xlsx')

In [3]:
#Travailler avec une copie du data Frame pour éviter de le rechager en cas d'êrreur
df = data.copy()

In [4]:
#Afficher les informations relatives au DF
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   AGE          918 non-null    int64  
 1   SEXE         918 non-null    object 
 2   TDT          918 non-null    object 
 3   PAR          918 non-null    int64  
 4   CHOLESTEROL  918 non-null    int64  
 5   GAJ          918 non-null    int64  
 6   ECG          918 non-null    object 
 7   FCMAX        918 non-null    int64  
 8   ANGINE       918 non-null    object 
 9   DEPRESSION   918 non-null    float64
 10  PENTE        918 non-null    object 
 11  CŒUR         918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [5]:
df.describe()

,AGE,PAR,CHOLESTEROL,GAJ,FCMAX,DEPRESSION,CŒUR
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


# Normalisation des variables avec la fonction std(). 

In [6]:
#Normalisation des variables quantitatives
for col in df.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df[col] = df[col]-df[col].mean()/df[col].std()


In [7]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,homme,AA,132.848903,287.182556,-0.551041,Normal,166.626568,Non,-0.831979,Ascendant,0
1,43.327036,femme,DNA,152.848903,178.182556,-0.551041,Normal,150.626568,Non,0.168021,Plat,1
2,31.327036,homme,AA,122.848903,281.182556,-0.551041,ST,92.626568,Non,-0.831979,Ascendant,0
3,42.327036,femme,ASY,130.848903,212.182556,-0.551041,Normal,102.626568,Oui,0.668021,Plat,1
4,48.327036,homme,DNA,142.848903,193.182556,-0.551041,Normal,116.626568,Non,-0.831979,Ascendant,0


In [8]:
#Encodage des variables qualitatives
for col in df.select_dtypes('object').columns:
    df[col] = df[col].astype('category').cat.codes

In [9]:
#Vérification des résultats
df.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,1,0,132.848903,287.182556,-0.551041,1,166.626568,0,-0.831979,0,0
1,43.327036,0,3,152.848903,178.182556,-0.551041,1,150.626568,0,0.168021,2,1
2,31.327036,1,0,122.848903,281.182556,-0.551041,2,92.626568,0,-0.831979,0,0
3,42.327036,0,1,130.848903,212.182556,-0.551041,1,102.626568,1,0.668021,2,1
4,48.327036,1,3,142.848903,193.182556,-0.551041,1,116.626568,0,-0.831979,0,0


In [10]:
#Séparer la variable cible (coeur) et les variables explicatives
y = df['CŒUR']
x = df.drop('CŒUR', axis = 1)

In [11]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [12]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [13]:
#Création d'un objet lr de la classe LogisticRegression
lr = LogisticRegression(solver ='newton-cg', random_state =1)

In [14]:
#Apprentissage du modèle
model = lr.fit(x_train, y_train)
#model = lr.fit(train_set.drop('CŒUR', axis = 1), train_set['CŒUR'])

In [15]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model.predict_proba(x_test)
predict_proba[:5,:]

array([[0.34770511, 0.65229489],
       [0.17508386, 0.82491614],
       [0.03708019, 0.96291981],
       [0.08135263, 0.91864737],
       [0.18111698, 0.81888302]])

In [16]:
#Application du modèle au données de test
y_pred = model.predict(x_test)
y_pred[:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [17]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[ 66,   8],
       [  8, 102]], dtype=int64)

In [18]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 85.32 % de bonne prédiction.

0.9130434782608695

In [19]:
#Sensibilité
recall = recall_score(y_test, y_pred)
recall
#conclusion : lorsqu'un individu est malade, nous avons 84.11 % de chance que le modèle le prédise.

0.9272727272727272

In [20]:
#Précision
precision = precision_score(y_test, y_pred)
precision
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 90 % de chace que cela soit vraie.

0.9272727272727272

In [21]:
#plus de détails avec classification report
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        74
           1       0.93      0.93      0.93       110

    accuracy                           0.91       184
   macro avg       0.91      0.91      0.91       184
weighted avg       0.91      0.91      0.91       184



In [22]:
#Modèle Trival
df['CŒUR'].value_counts()/df.shape[0]
#Conclusion : si nous nous basons sur le hasard pour faire nos prédiction, nous aurons 55.37 % de bonne prédiction.

1    0.553377
0    0.446623
Name: CŒUR, dtype: float64

In [23]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model.score(x_train, y_train)

0.8460490463215259

In [24]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model.score(x_test, y_test)

0.9130434782608695

In [25]:
df

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,1,0,132.848903,287.182556,-0.551041,1,166.626568,0,-0.831979,0,0
1,43.327036,0,3,152.848903,178.182556,-0.551041,1,150.626568,0,0.168021,2,1
2,31.327036,1,0,122.848903,281.182556,-0.551041,2,92.626568,0,-0.831979,0,0
3,42.327036,0,1,130.848903,212.182556,-0.551041,1,102.626568,1,0.668021,2,1
4,48.327036,1,3,142.848903,193.182556,-0.551041,1,116.626568,0,-0.831979,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,39.327036,1,2,102.848903,262.182556,-0.551041,1,126.626568,0,0.368021,2,1
914,62.327036,1,1,136.848903,191.182556,0.448959,1,135.626568,0,2.568021,2,1
915,51.327036,1,1,122.848903,129.182556,-0.551041,1,109.626568,1,0.368021,2,1
916,51.327036,0,0,122.848903,234.182556,-0.551041,0,168.626568,0,-0.831979,2,1


# DecisionTree

In [26]:
data

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


In [27]:
#faire une copie du dataFrame de base pour avoir un autre sur lequel je pourais appliquer mon arbre de décision 

In [28]:
df_AD = data.copy()

In [29]:
df_AD

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,40,homme,AA,140,289,0,Normal,172,Non,0.0,Ascendant,0
1,49,femme,DNA,160,180,0,Normal,156,Non,1.0,Plat,1
2,37,homme,AA,130,283,0,ST,98,Non,0.0,Ascendant,0
3,48,femme,ASY,138,214,0,Normal,108,Oui,1.5,Plat,1
4,54,homme,DNA,150,195,0,Normal,122,Non,0.0,Ascendant,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,homme,AT,110,264,0,Normal,132,Non,1.2,Plat,1
914,68,homme,ASY,144,193,1,Normal,141,Non,3.4,Plat,1
915,57,homme,ASY,130,131,0,Normal,115,Oui,1.2,Plat,1
916,57,femme,AA,130,236,0,LVH,174,Non,0.0,Plat,1


# Normalisation des variables avec la fonction std().

In [30]:
df_AD.describe()

,AGE,PAR,CHOLESTEROL,GAJ,FCMAX,DEPRESSION,CŒUR
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [31]:
#Normalisation des variables quantitatives
for col in df_AD.drop('CŒUR', axis =1).select_dtypes(np.number).columns:
    df_AD[col] = df_AD[col]-df_AD[col].mean()/df_AD[col].std()


In [32]:
#Vérification des résultats
df_AD.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,34.327036,homme,AA,132.848903,287.182556,-0.551041,Normal,166.626568,Non,-0.831979,Ascendant,0
1,43.327036,femme,DNA,152.848903,178.182556,-0.551041,Normal,150.626568,Non,0.168021,Plat,1
2,31.327036,homme,AA,122.848903,281.182556,-0.551041,ST,92.626568,Non,-0.831979,Ascendant,0
3,42.327036,femme,ASY,130.848903,212.182556,-0.551041,Normal,102.626568,Oui,0.668021,Plat,1
4,48.327036,homme,DNA,142.848903,193.182556,-0.551041,Normal,116.626568,Non,-0.831979,Ascendant,0


In [33]:
#Encodage des variables qualitatives dans une variable le

le = LabelEncoder()

In [34]:
#creer une variable df_encoded qui va stocker notre dataframe encoder
df_encoded=df.iloc[:,0:12]

In [35]:
#recodons par exemple le 'PENTE'
le.fit_transform(df_encoded["PENTE"])



array([0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2,
       0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0,
       2, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 0, 2, 0, 2, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0,
       2, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 2,
       2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0,
       0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 0,
       2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2,
       2, 0, 2, 0, 2, 1, 2, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2,
       2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2,

In [36]:
#Dans une fonction application cette modification à tout notre DataFrame
for i in df_encoded:
    df_encoded[i]=le.fit_transform(df_encoded[i])


In [37]:
df_encoded.head()

,AGE,SEXE,TDT,PAR,CHOLESTEROL,GAJ,ECG,FCMAX,ANGINE,DEPRESSION,PENTE,CŒUR
0,12,1,0,41,147,0,1,98,0,10,0,0
1,21,0,3,55,40,0,1,82,0,20,2,1
2,9,1,0,31,141,0,2,25,0,10,0,0
3,20,0,1,39,72,0,1,34,1,25,2,1
4,26,1,3,49,53,0,1,48,0,10,0,0


In [38]:
# jeu de fonctionnalités
x=df_encoded.iloc[:,0:11]
#jeu d'étiquettes
y=df_encoded.iloc[:,11]

In [39]:
#Subdivision du jeu de données en apprentissage et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state =1)
#train_set, test_set = train_test_split(df,test_size =0.2, random_state =1)

In [40]:
#Vérifier le résultat
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(734, 11) (184, 11) (734,) (184,)


In [41]:
#Création d'un objet model de la classe DecisionTreeClassifier
model_AD=DecisionTreeClassifier(criterion='gini')
#Gini est une formule mathematique de la fonction CART

In [42]:
#Apprentissage du modèle
model_AD.fit(x_train,y_train)

DecisionTreeClassifier()

In [43]:
#Probabilité d'appartenance à l'une des classes
predict_proba = model_AD.predict_proba(x_test)
predict_proba[:5,:]

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])

In [44]:
#Application du modèle au données de test
y_pred = model_AD.predict(x_test)
y_pred[:5]

array([1, 0, 1, 1, 0], dtype=int64)

In [45]:
#matrice de confusion
mc = confusion_matrix(y_test, y_pred)
mc

array([[61, 13],
       [27, 83]], dtype=int64)

In [46]:
#Taux de bonnes prédictions
accuracy = accuracy_score(y_test, y_pred)
accuracy
#Conclusion : le modèle produit 81 % de bonne prédiction.

0.782608695652174

In [47]:
#Sensibilité
recall_AD = recall_score(y_test, y_pred)
recall_AD
#conclusion : lorsqu'un individu est malade, nous avons 79 % de chance que le modèle le prédise.

0.7545454545454545

In [48]:
#Précision
precision_AD = precision_score(y_test, y_pred)
precision_AD
#Conclusion : lorsque le modèle prédit qu'un individu est malade, nous avons 87,9 % de chace que cela soit vraie.

0.8645833333333334

In [49]:
#plus de détails avec classification report
cr_AD = classification_report(y_test, y_pred)
print(cr_AD)

              precision    recall  f1-score   support

           0       0.69      0.82      0.75        74
           1       0.86      0.75      0.81       110

    accuracy                           0.78       184
   macro avg       0.78      0.79      0.78       184
weighted avg       0.80      0.78      0.78       184



In [50]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données d'entrainement)
model_AD.score(x_train, y_train)

1.0

In [51]:
#Score d'entrainement (le taux de bonne prédiction calculé sur la base des données de test)
model_AD.score(x_test, y_test)

0.782608695652174

# Comparaison des scores entre la Régression Linéaire et L'Arbre de Décision

In [55]:
if model.score(x_test, y_test) > model_AD.score(x_test, y_test):
    print("Sur la base des données de test, le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("Sur la base des données de test, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")
print()
print("Et")
print()

if model.score(x_train, y_train) > model_AD.score(x_train, y_train):
    print("sur la base des données d'entrainement, le resultat de la Régression Logistique est supérieure à celui de l'Arbre de Décision")
else:
    print("sur la base des données d'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision")
print()
print("Finalement")
print()

if model.score(x_train, y_train) < model_AD.score(x_train, y_train) and model.score(x_test, y_test) < model_AD.score(x_test, y_test):
    print("L'arbre de Décisin produit un bon résultat par raport à celui de la Régression Log")
else:
    print("L'arbre de Décisin produit un mauvais résultat par raport à celui de la Régression Log")


Sur la base des données de test, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision

Et

sur la base des données d'entrainement, le resultat de la Régression Logistique est inférieure à celui de l'Arbre de Décision

Finalement

L'arbre de Décisin produit un bon résultat par raport à celui de la Régression Log
